<a href="https://colab.research.google.com/github/ThaiNguyenLongHai/Hai/blob/main/AI_report_flower.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import os
from keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Input
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def save_data(root_folder):

    dest_size = (224, 224)
    print("XU lY ANH")

    images = []
    labels = []

    # Lặp qua các thư mục con con trong thư mục gốc
    for folder in os.listdir(root_folder):
        if folder!='.DS_Store':
            print("Folder =",folder)
            # Lặp qua các file trong từng thư mục con  chứa các ảnh
            for file in os.listdir(root_folder+'/'+folder):
                if file!='.DS_Store':
                    # print("File=", file)
                    images.append(cv.resize(cv.imread(root_folder + '/' + folder +"/" + file), dsize=(224, 224)))
                    labels.append(folder)

    images = np.array(images)
    labels = np.array(labels)

    encoder = LabelBinarizer()
    labels = encoder.fit_transform(labels)
    print(labels)

    file = open('/content/drive/MyDrive/Colab Notebooks/flowers.data', 'wb')
    # dump information to that file
    pickle.dump((images,labels), file)
    # close the file
    file.close()

save_data('/content/drive/MyDrive/Colab Notebooks/flower')

In [ ]:
def load_data():
    file = open('/content/drive/MyDrive/Colab Notebooks/flowers.data', 'rb')

    # dump information to that file
    (pixels, labels) = pickle.load(file)

    # close the file
    file.close()

    print(pixels.shape)
    print(labels.shape)

    return pixels, labels

images, labels = load_data()



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state = 10)
X = X_test

In [ ]:
X_train = X_train.astype('float32')/255.0
X_test = X_test.astype('float32')/255.0

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3,3), strides = (1,1), padding='same', activation='relu', input_shape=(224, 224, 3)))
model.add(Conv2D(32, (3,3), strides = (1,1), padding='same', activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64, (3,3), strides = (1,1), padding='same', activation='relu'))
model.add(Conv2D(64, (3,3), strides = (1,1), padding='same', activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(128, (3,3), strides = (1,1), padding='same', activation='relu'))
model.add(Conv2D(128, (3,3), strides = (1,1), padding='same', activation='relu'))
model.add(MaxPooling2D((2,2)))


model.add(Conv2D(256, (3,3), strides = (1,1), padding='same', activation='relu'))
model.add(Conv2D(256, (3,3), strides = (1,1), padding='same', activation='relu'))
model.add(Conv2D(256, (3,3), strides = (1,1), padding='same', activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64, (3,3), strides = (1,1), padding='same', activation='relu'))
model.add(Conv2D(64, (3,3), strides = (1,1), padding='same', activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0002), metrics=['accuracy'])

In [ ]:
f = ImageDataGenerator(rotation_range=20,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')

In [ ]:
t = model.fit_generator(f.flow(X_train, y_train, batch_size=64), 
                        epochs=30, validation_data= f.flow(X_test,y_test, batch_size=64), verbose=1)

In [ ]:
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 30), t.history["loss"], label="train_loss")
plt.plot(np.arange(0, 30), t.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 30), t.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, 30), t.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Acc")
plt.xlabel("No.Epoch")
plt.ylabel("Loss/Accuracy")
plt.legend()

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/model_flower.h5')

In [ ]:
class_name = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']
image = X[400]
plt.style.use('default')
c = cv.cvtColor(image, cv.COLOR_BGR2RGB)
plt.imshow(c)
plt.show()

img = X_test[400]
img = np.expand_dims(img, axis = 0)
result = model.predict(img)
# print kết quả 
print("The picture: ", class_name[np.argmax(result[0])], (result[0]))
